In [2]:
import pandas as pd
import zipfile

In [3]:
with zipfile.ZipFile("crashes.csv.zip") as z1:
    with z1.open("crashes.csv") as crashes:
        df_crashes = pd.read_csv(crashes)

df_crashes.head()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,JC199149,NaN,03/25/2019 02:43:00 PM,30.0,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,0.0,1.0,2.0,0.0,14.0,2.0,3.0,41.884547,-87.641201,POINT (-87.64120093714 41.884547224337)
1,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,JB422857,NaN,09/05/2018 08:40:00 AM,30.0,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,...,0.0,0.0,2.0,0.0,8.0,4.0,9.0,41.968562,-87.740659,POINT (-87.740659314632 41.968562453871)
2,0115ade9a755e835255508463f7e9c4a9a0b47e9304238...,JF318029,NaN,07/15/2022 12:45:00 AM,30.0,UNKNOWN,UNKNOWN,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,...,0.0,0.0,2.0,0.0,0.0,6.0,7.0,41.886336,-87.716203,POINT (-87.716203130599 41.886336409761)
3,017040c61958d2fa977c956b2bd2d6759ef7754496dc96...,JF324552,NaN,07/15/2022 06:50:00 PM,30.0,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,...,0.0,0.0,2.0,0.0,18.0,6.0,7.0,41.925111,-87.667997,POINT (-87.667997321599 41.925110815832)
4,78eee027ec3dcc85d36c9e3fdae4729dcc56440105d65b...,JB291672,NaN,06/03/2018 05:00:00 PM,30.0,NO CONTROLS,NO CONTROLS,CLEAR,UNKNOWN,PARKED MOTOR VEHICLE,...,0.0,0.0,1.0,0.0,17.0,1.0,6.0,41.910758,-87.731389,POINT (-87.731388754145 41.910757551599)


In [127]:
with zipfile.ZipFile("people.csv.zip") as z1:
    with z1.open("people.csv") as people:
        df_people = pd.read_csv(people)

df_people.head()

/var/folders/g7/g9fxssr51yd508bw8xr5xtcw0000gn/T/ipykernel_23799/706441852.py:3: DtypeWarning: Columns (20,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_people = pd.read_csv(people)


,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,09/28/2019 03:30:00 AM,NaN,CHICAGO,IL,60651,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,04/13/2020 10:50:00 PM,NaN,CHICAGO,IL,60620,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,HY484534,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,HY484750,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,HY484778,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [4]:
with zipfile.ZipFile("vehicles.csv.zip") as z1:
    with z1.open("vehicles.csv") as vehicles:
        df_vehicles = pd.read_csv(vehicles)

df_vehicles.head()

/var/folders/g7/g9fxssr51yd508bw8xr5xtcw0000gn/T/ipykernel_1273/2186791932.py:3: DtypeWarning: Columns (0,19,21,40,41,42,44,48,49,50,55,58,59,60,61,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vehicles = pd.read_csv(vehicles)


,CRASH_UNIT_ID,CRASH_RECORD_ID,RD_NO,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,1554880,91a5d08b2b701f2d37cbb52ecdbeb09579bc7f2ebc60b3...,JG223284,04/14/2023 02:05:00 PM,1.0,DRIVER,NaN,1478881.0,NaN,FORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,1.0,DRIVER,NaN,834816.0,NaN,HONDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,749949,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,2.0,PARKED,NaN,834819.0,NaN,TOYOTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,749950,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,3.0,PARKED,NaN,834817.0,NaN,GENERAL MOTORS CORPORATION (GMC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1554881,91a5d08b2b701f2d37cbb52ecdbeb09579bc7f2ebc60b3...,JG223284,04/14/2023 02:05:00 PM,2.0,DRIVER,1.0,1478892.0,NaN,ISUZU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Since the vehicles csv file has many columns, we can specify which columns to include into our analysis.

In [5]:
vehicles_cols = ['CRASH_RECORD_ID', 'UNIT_NO', 'UNIT_TYPE', 'VEHICLE_ID', 'MAKE', 'MODEL', 'VEHICLE_YEAR', 
                'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE', 'TRAVEL_DIRECTION', 'MANEUVER', 'FIRST_CONTACT_POINT']


df_vehicles = df_vehicles[vehicles_cols]

df_vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747121 entries, 0 to 747120
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CRASH_RECORD_ID      747117 non-null  object 
 1   UNIT_NO              747117 non-null  float64
 2   UNIT_TYPE            745326 non-null  object 
 3   VEHICLE_ID           721931 non-null  float64
 4   MAKE                 721931 non-null  object 
 5   MODEL                721892 non-null  object 
 6   VEHICLE_YEAR         584676 non-null  float64
 7   VEHICLE_DEFECT       721931 non-null  object 
 8   VEHICLE_TYPE         721931 non-null  object 
 9   VEHICLE_USE          721931 non-null  object 
 10  TRAVEL_DIRECTION     721931 non-null  object 
 11  MANEUVER             721931 non-null  object 
 12  FIRST_CONTACT_POINT  720719 non-null  object 
dtypes: float64(3), object(10)
memory usage: 74.1+ MB


Completing inner merge on the `CRASH_RECORD_ID`.

In [6]:
df_crashes_vehicles = pd.merge(df_crashes, df_vehicles,
                                on="CRASH_RECORD_ID",
                                how="inner")

df_crashes_vehicles.head()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,VEHICLE_ID,MAKE,MODEL,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,FIRST_CONTACT_POINT
0,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,JC199149,NaN,03/25/2019 02:43:00 PM,30.0,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,582401.0,CHEVROLET,CRUZE,2017.0,NONE,PASSENGER,RIDESHARE SERVICE,W,TURNING LEFT,FRONT-LEFT
1,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,JC199149,NaN,03/25/2019 02:43:00 PM,30.0,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,582411.0,"TOYOTA MOTOR COMPANY, LTD.",CAMRY,2010.0,NONE,PASSENGER,PERSONAL,W,STRAIGHT AHEAD,FRONT-RIGHT
2,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,JB422857,NaN,09/05/2018 08:40:00 AM,30.0,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,...,450562.0,"TOYOTA MOTOR COMPANY, LTD.",4RUNNER,2002.0,NONE,SPORT UTILITY VEHICLE (SUV),PERSONAL,N,STRAIGHT AHEAD,FRONT
3,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,JB422857,NaN,09/05/2018 08:40:00 AM,30.0,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,...,450569.0,"TOYOTA MOTOR COMPANY, LTD.",CAMRY,2014.0,NONE,PASSENGER,PERSONAL,SE,STRAIGHT AHEAD,REAR-RIGHT
4,78eee027ec3dcc85d36c9e3fdae4729dcc56440105d65b...,JB291672,NaN,06/03/2018 05:00:00 PM,30.0,NO CONTROLS,NO CONTROLS,CLEAR,UNKNOWN,PARKED MOTOR VEHICLE,...,386031.0,UNKNOWN,UNKNOWN,NaN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,UNKNOWN,UNKNOWN/NA,UNKNOWN


In [10]:
df_crashes_vehicles["DAMAGE"].isna().sum()

2

In [131]:
# there are 2 rows that had missing values for many columns
# so drop these two rows
df_crashes_vehicles = df_crashes_vehicles.loc[df_crashes_vehicles["DAMAGE"].notna()]

In [11]:
df_crashes_vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444133 entries, 0 to 444132
Data columns (total 61 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                444133 non-null  object 
 1   RD_NO                          444018 non-null  object 
 2   CRASH_DATE_EST_I               36484 non-null   object 
 3   CRASH_DATE                     444133 non-null  object 
 4   POSTED_SPEED_LIMIT             444133 non-null  float64
 5   TRAFFIC_CONTROL_DEVICE         444133 non-null  object 
 6   DEVICE_CONDITION               444133 non-null  object 
 7   WEATHER_CONDITION              444133 non-null  object 
 8   LIGHTING_CONDITION             444133 non-null  object 
 9   FIRST_CRASH_TYPE               444131 non-null  object 
 10  TRAFFICWAY_TYPE                444131 non-null  object 
 11  LANE_CNT                       239162 non-null  float64
 12  ALIGNMENT                     

In [13]:
df_crashes_vehicles["CRASH_TYPE"].value_counts()

CRASH_TYPE
NO INJURY / DRIVE AWAY              336980
INJURY AND / OR TOW DUE TO CRASH    107151
Name: count, dtype: int64

In [14]:
df_crashes_vehicles["MOST_SEVERE_INJURY"].value_counts()

MOST_SEVERE_INJURY
NO INDICATION OF INJURY     384406
NONINCAPACITATING INJURY     32751
REPORTED, NOT EVIDENT        17959
INCAPACITATING INJURY         7803
FATAL                          476
Name: count, dtype: int64

In [15]:
df_crashes_vehicles["DAMAGE"].value_counts()

DAMAGE
OVER $1,500      254807
$501 - $1,500    130693
$500 OR LESS      58631
Name: count, dtype: int64